In [1]:
#iterate over all rlocs to get average interaction strenght
#this is done twice to create ONE file for BRAF and one file for RAS
#those r then named: AverageRloc_ForCorrelationPlot_<braf/ras>.txt

#and max/min values
import sys
sys.path.insert(0, '../../helperScripts/')
from helperFunctions import *
import scmra
import os
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
import pandas as pd
from datetime import datetime
import pickle
import seaborn as sns; sns.set_theme()
from matplotlib.pyplot import figure
import scmra    



In [8]:
def get_population_list_from_string(string):
   popList = []
   x = string.split(",")
   for pop in x:
      if pop == "": 
         continue
      else:
         popList.append(pop)
   return(popList)

def calculate_diff_dict():
   dict = {}
   dict['braf'] = ((rloc_true - rloc_braf)**2)**0.5
   dict['ras'] = ((rloc_true - rloc_ras)**2)**0.5

   return(dict)

def extract_average_rloc_differences(pickleFile, outputFile,):

   result = pickle.load(open(pickleFile, 'rb'))

   # for every combination of
   # NOISE CELLNUM REPEAT of CNR
   headers = result.header
   resultFrame = pd.DataFrame(columns=headers)
   #add all the existing parameters
   resultNumber = len(result.parameter)
   for i in range(resultNumber):
      params = result.parameter[i]
      df_length = len(resultFrame)
      resultFrame.loc[df_length] = params
   cnrData = resultFrame[resultFrame["MODELTYPE"] == "CNR"]
   
   #get noise cellnum and repeats for CNR
   noise = 0.2
   cellnum = 250
   repeatLevels = cnrData["REPEAT"].unique()
   theta = 0.02 #this is anyways only 0.02 for CNR or 'NO_THETA' for MRA
   mutations = ["ras", "braf"]
   keyList = ["MODELTYPE", "NOISE", "CELLNUM", "POPULATIONS","REPEAT","THETA","EDGES"]
   
   for mutation in mutations:
      if(mutation == "ras"):
         populationLevel = "wt,ras"
      elif(mutation == "braf"):
         populationLevel = "wt,braf"

      # Create the pandas DataFrame
      df = pd.DataFrame(columns=[   "PROTEIN_A", "PROTEIN_B", "RLOC_RECONSTRUCTED", "TYPE",  "RLOC_TRUE", "MIN" , "MAX"])

      averageRlocWt = rloc_true.copy()
      minRlocWt = rloc_true.copy()
      maxRlocWt = rloc_true.copy()
      averageRlocMut = rloc_true.copy()
      minRlocMut = rloc_true.copy()
      maxRlocMut = rloc_true.copy()

      for col in rloc_true.columns:
         averageRlocWt[col].values[:] = 0
         minRlocWt[col].values[:] = 10
         maxRlocWt[col].values[:] = -10
         averageRlocMut[col].values[:] = 0
         minRlocMut[col].values[:] = 10
         maxRlocMut[col].values[:] = -10

      for repeat in repeatLevels:
            valueList = ["CNR", noise, cellnum, populationLevel, repeat, theta, 13]
            popList = get_population_list_from_string(populationLevel) # list of populations like ["wt", "braf"]
            if(len(popList) != 2): 
               print("NOT EXACTLY 2 POPULATIONS FOR MRA -> SKIP MRA RLOC DIFF CALCUALTION\n")
               continue
            if(not "wt" in popList and not "WT" in popList): 
               print("NO wildtype <wt> in population list (needed for rloc Diff) -> SKIP MRA RLOC DIFF CALCUALTION \n")
               continue

            resultObject, params = result.getResult(keyList, valueList) #the result from the simulation
            if(resultObject == None): continue

            vars_lst = [var for var in resultObject.vardict if var.startswith('IDev')]
            vars_vals = [resultObject.vardict[v] for v in vars_lst]
            lengthIdcs = (np.count_nonzero(vars_vals))

            assert(len(popList) == 2)
            rloc_list = {}
            #we assign population by wt, braf, ras. Those MUST BE LOWER CASE NAMES.
            #just in case convert to lower case beforehand
            lowerPopList = []
            for popTmp in popList:
               lowerPopList.append(popTmp.lower())
            popList = lowerPopList
            for popStringIdx in range(len(popList)):
               popString = popList[popStringIdx]
               rloc_tmp = resultObject.rloc[popString]
               rloc_tmp = reorder_rloc(rloc_tmp)
               rloc_list[popList[popStringIdx]] = (rloc_tmp)

            # get all rloc differences and write one line per difference
            columns = (rloc_true.columns)
            rows = (rloc_true.index)

            rlocMut = rloc_list[mutation]
            rlocWt = rloc_list["wt"]
            print("============")
            print(rlocMut)
            print(rlocWt)
            for col in columns:
               for row in rows:

                  #for average just sum up values first
                  averageRlocMut.at[row, col] += rlocMut.at[row, col].copy()
                  minRlocMut.at[row, col] = min([rlocMut.at[row, col], minRlocMut.at[row, col]]).copy()
                  maxRlocMut.at[row, col] = max([rlocMut.at[row, col], maxRlocMut.at[row, col]]).copy()

                  averageRlocWt.at[row, col] += rlocWt.at[row, col].copy()
                  minRlocWt.at[row, col] = min([rlocWt.at[row, col], minRlocWt.at[row, col]]).copy()
                  maxRlocWt.at[row, col] = max([rlocWt.at[row, col], maxRlocWt.at[row, col]]).copy()
      
      #create final dataframe
      columns = (rloc_true.columns)
      rows = (rloc_true.index)
      print(rloc_true)

      if(mutation == "ras"):
         rloc_mut = rloc_ras
      elif(mutation == "braf"):
         rloc_mut = rloc_braf

      for col in columns:
         for row in rows: 
            # dataframe struture is: columns=[   PROTEIN_A PROTEIN_B RLOC_RECONSTRUCTED TYPE  RLOC_TRUE]
            rowList = [row, col, averageRlocWt.at[row, col]/len(repeatLevels), "wt", rloc_true.at[row, col], minRlocWt.at[row, col], maxRlocWt.at[row, col]]
            df.loc[len(df)] = rowList

            rowList = [row, col, averageRlocMut.at[row, col]/len(repeatLevels), mutation, rloc_mut.at[row, col], minRlocMut.at[row, col], maxRlocMut.at[row, col]]
            df.loc[len(df)] = rowList

      #finally print the dataframe
      df.to_csv("./bin/" + outputFile + "_" + mutation + ".txt", index=False, sep="\t")

In [9]:
pickleFile = "/DATA/t.stohn/MRA/scMRA-analysis/notebooks/paperAnalyses/2_CNRAnalysis/bin/SIMULATION_CnrOverMra_NoiseRange_CellNum250_04-19-2023|20:59:39.pickle"
result = pickle.load(open(pickleFile, 'rb'))

print(rloc_true)
print(rloc_ras)
extract_average_rloc_differences(pickleFile, "AverageRloc_ForCorrelationPlot")

             Sos       Ras      Raf1       Mek      Erk    P90Rsk      PI3K  \
Sos    -1.000000  0.000000  0.000000  0.000000  0.00000 -0.807052  0.000000   
Ras     0.999877 -1.000000  0.000000  0.000000  0.00000  0.000000  0.000000   
Raf1    0.000000  1.001393 -1.000000  0.000000  0.00000  0.000000  0.000000   
Mek     0.000000  0.000000  0.087174 -1.000000  0.00000  0.000000  0.000000   
Erk     0.000000  0.000000  0.000000  0.991678 -1.00000  0.000000  0.000000   
P90Rsk  0.000000  0.000000  0.000000  0.000000  0.95669 -1.000000  0.000000   
PI3K    0.000000  0.000176  0.000000  0.000000  0.00000  0.000000 -1.000000   
Akt     0.000000  0.000000  0.000000  0.000000  0.00000  0.000000  0.396081   
C3G     0.000000  0.000000  0.000000  0.000000  0.00000  0.000000  0.000000   
Rap1    0.000000  0.000000  0.000000  0.000000  0.00000  0.000000  0.000000   
bRaf    0.000000  0.165986  0.000000  0.000000  0.00000  0.000000  0.000000   

             Akt      C3G      Rap1      bRaf  
Sos